# Working with a database

This week's assignment has a few basic steps.  First, we're going to pull some data down off the internet and store it into our MySQL database.  Make sure that you use your username as part of the table name as show in the examples so that you don't create a problem for other students.

Then, we'll merge that with some data already in the database and calculate a few results.  When it comes to calculating the results, you can do so either with SQL or with Pandas operations.


## PART 1: Setup your database connection and table name

In the code below, change the value of the variable `MYTABLE` to use your own username rather that `'pboal'`

You can then use `MYTABLE` in the rest of your code to reference that table name.

In [52]:
import getpass
import os
import pymysql
from sqlalchemy import create_engine
import pandas as pd

host = 'slucor2022-instance-1.cgdcoitnku0k.us-east-1.rds.amazonaws.com'
port = '3306'
user = 'slucor2022'
password = 'SLUcor2022'
database = 'hds5210'

conn = create_engine('mysql+pymysql://' + 
                     user + ':' + 
                     password + '@' + 
                     host + '/' + 
                     database, echo=False)

vmuni = getpass.getuser().split('-')[1]
vmuni
MYTABLE= vmuni + '_data'



In [53]:
assert(MYTABLE != 'pboal')
assert(conn.connect())

## PART 2: Bring in outside data

Grab the data from this URL and put it into a database table named with your `username_data`.

https://opendata.arcgis.com/api/v3/datasets/ed0bc689bde246b18835c7529ba4efb4_0/downloads/data?format=csv&spatialRefId=4326

By the end of your cell, the table should be created.  The tests are going to verify that the table exists and looks right.

In [54]:
ehr_data = pd.read_csv('Electronic_Health_Record_EHR_Incentive_Program_Payments_to_Eligible_Hospitals.csv')
ehr_data.to_sql(MYTABLE + '_data', conn, index = True, if_exists = 'replace')

In [55]:
vmuni = pd.read_sql_query('SELECT * FROM ' + MYTABLE, conn)
vmuni

,X,Y,OBJECTID,Provider_Name,NPI,CCN,Business_Street_Address,Business_City,Business_County,Business_ZIP_Code,...,Payment_Year_Number,Program_Type,Medicaid_EP_Hospital_Type,total_payments,Last_Payment_Criteria,Recent_Disbursement_Amount,Latitude,Longitude,Last_Program_Year,Last_Payment_Year
0,-124.142009,40.783559,1,ST JOSEPH HEALTH NORTHERN CALIFORNIA LLC,1609858950,50006,2700 Dolbeer St,Eureka,Humboldt,95501,...,4,Medicare/Medicaid,Acute Care Hospitals,1530950.70,MU,153095.07,40.783559,-124.142009,2015,2016
1,-122.086674,37.632915,2,HAYWARD SISTERS HOSPITAL,1942298153,50002,27200 Calaroga Ave,Hayward,Alameda,94545,...,4,Medicare/Medicaid,Acute Care Hospitals,3245920.28,MU,324592.03,37.632915,-122.086674,2015,2016
2,-122.295861,38.325402,3,ST JOSEPH HEALTH NORTHERN CALIFORNIA LLC,1235218785,50009,1000 Trancas St,Napa,Napa,94558,...,4,Medicare/Medicaid,Acute Care Hospitals,1262015.89,MU,126201.59,38.325402,-122.295861,2015,2016
3,-120.766467,38.350920,4,SUTTER VALLEY HOSPITALS,1447494323,50014,200 Mission Blvd,Jackson,Amador,95642,...,4,Medicare/Medicaid,Acute Care Hospitals,865130.13,MU,86513.01,38.350920,-120.766467,2017,2019
4,-122.435820,37.769049,5,Sutter Bay Hospitals,1659439834,50008,CASTRO AND DUBOCE,San Francisco,San Francisco,94114,...,4,Medicare/Medicaid,Acute Care Hospitals,984465.12,MU,98446.51,37.769049,-122.435820,2016,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,-122.173214,37.436328,319,LUCILE SALTER PACKARD CHILDREN'S HOSPITAL AT S...,1467442749,53305,"725 Welch Rd, mail code 5500",Palo Alto,Santa Clara,94304,...,4,Medicaid,Children's Hospitals,5630849.63,MU,563084.96,37.436328,-122.173214,2017,2019
319,-118.186632,33.808293,320,LONG BEACH MEMORIAL MEDICAL,1477596583,53309,2801 Atlantic Ave,Long Beach,Los Angeles,90806,...,4,Medicaid,Children's Hospitals,7951873.30,MU,795187.33,33.808293,-118.186632,2015,2016
320,-121.455450,38.553012,321,SHRINERS HOSPITALS FOR CHILDREN,1962530451,53311,2425 Stockton Blvd,Sacramento,Sacramento,95817,...,4,Medicaid,Children's Hospitals,2184269.03,MU,218426.90,38.553012,-121.455450,2018,2020
321,-118.286401,34.067178,322,SHRINERS HOSPITALS FOR CHILDREN,1508908591,53310,3160 Geneva Street,Los Angeles,Los Angeles,90020,...,1,Medicaid,Children's Hospitals,427640.22,AIU,427640.22,34.067178,-118.286401,2016,2017


In [56]:
dxyz = pd.read_sql_query('SELECT * FROM ' + MYTABLE, conn)
assert(dxyz.shape == (323,22))
assert(list(dxyz.columns) == ['index', 'X', 'Y', 'OBJECTID', 'Provider_Name', 'NPI', 'CCN',
       'Business_Street_Address', 'Business_City', 'Business_County',
       'Business_ZIP_Code', 'Business_State_Territory', 'Payment_Year_Number',
       'Program_Type', 'Medicaid_EP_Hospital_Type', 'total_payments',
       'Last_Payment_Criteria', 'Recent_Disbursement_Amount', 'Latitude',
       'Longitude', 'Last_Program_Year', 'Last_Payment_Year'])

#data.shape assertion seems to be off for some reason

AssertionError: 

## PART 3: Combine with other data in the database

In the database, there is an existing table called `population`.  We want to merge the DHCS datafile loaded above with the population data available in this other database table  The tables can be merged on `MYTABLE`'s `Business_ZIP_Code` field and `population`'s `Zip` field.

Note that not all `Zip_Codes` from your downloaded file have to be in the `population` table.  If they aren't, then I want you to eliminate the non-matching records.  That is, only keep the records that have a matching ZIP code in both sets of data.

Answer the question:
Which providers are located in the zipcode with the largest population?

Put your answer in the form `answer = ['a', 'list', 'of', 'NPI', 'like', '1593042103]`

In [75]:
data_join = pd.read_sql_query("""
  SELECT h.* , p.population
  FROM' + MYTABLE + 'h JOIN
    population p ON h.Business_ZIP_Code = p.zip
  """,conn)
data_join

ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '' + MYTABLE + 'h JOIN\n    population p ON h.Business_ZIP_Code = p.zip' at line 2")
[SQL: 
  SELECT h.* , p.population
  FROM' + MYTABLE + 'h JOIN
    population p ON h.Business_ZIP_Code = p.zip
  ]
(Background on this error at: http://sqlalche.me/e/f405)

In [76]:
assert(type(answer) == list)
assert(answer == ['1194016923'])


AssertionError: 

## PART 4: Total by hospital type

This is a multistep process:
* From our downloaded data file, compute the `Total payments` per ZIP code and Medicaid EP Hospital Type.
* Then merge that with the `population` data to compute a `Total payments` per person.
* Then average that across all of the `Medicaid EP Hospital Types` to get an average per persona payment for these type of hospital.

Your answer should be in structure of a data frame with at least two columns:
* Medicaid_EP_Hospital_Type
* Avg_Pay_per_Capita

## First using SQL

In [77]:
payment = pd.read_sql_query("""
  SELECT a.Medicaid_EP_Hospital_type, sum(a.total_payments) as payments, a.Business_ZIP_Code
  FROM
    vineethmunigati_data a
  GROUP BY
     a.Medicaid_EP_Hospital_Type, a.Business_ZIP_Code
""", conn)
payment.to_sql('payment_data', conn, index = False)
answer = pd.read_sql_query("""
  SELECT a.Medicaid_EP_Hospital_Type, AVG(a.payments/p.Population) as Avg_Pay_per_Capita
  from
    population p JOIN
    payment_data a ON p.Zip = a.Business_ZIP_Code
  Group by
    a.Medicaid_EP_Hospital_Type
""", conn)

ValueError: Table 'payment_data' already exists.

In [78]:
assert(type(answer) == pd.core.frame.DataFrame)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Acute Care Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 106.583)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Children\\'s Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 329.551)


In [79]:
answer

,Medicaid_EP_Hospital_Type,Avg_Pay_per_Capita
0,Acute Care Hospitals,106.582669
1,Children's Hospitals,329.550946


## Now using Pandas instead of SQL

In [82]:
payment_zip = pd.pivot_table(data_join,
            index = ['Business_ZIP_Code', 'Medicaid_EP_Hospital_Type'],
                            values = 'total_payments',
                            aggfunc = np.sum).reset_index()
total = payment_zip.merge(tables['population'], left_on = 'Business_ZIP_Code', right_on = 'Zip')
total['per_capita'] = (total['total_payments']/total['Population'])
avg_per_capita = total.groupby(['Medicaid_EP_Hospital_Type'])['per_capita'].agg(Avg_Pay_per_Capita = ('mean')).reset_index()
avg_per_capita

NameError: name 'data_join' is not defined



---



## Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Follow the instruction on the prompt below to either ssave and submit your work, or continue working.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

---

In [24]:
a=input('''
Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

''')

if a=='yes':
    !git pull
    !git add week15_assignment.ipynb
    !git commit -a -m "Submitting the week 15 programming assignment"
    !git push
else:
    print('''
    
OK. We can wait.
''')


Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

 yes


Already up to date.
[main f9ad580] Submitting the week 15 programming assignment
 2 files changed, 959 insertions(+), 18 deletions(-)
 create mode 100644 week15/week15_assignment.ipynb
Counting objects: 6, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 11.02 KiB | 5.51 MiB/s, done.
Total 6 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
remote: This repository moved. Please use the new location:
remote:   git@github.com:VineethMunigati/hds5210-2022.git
To github.com:vineethmunigati/hds5210-2022.git
   cf2b8a8..f9ad580  main -> main
